<a href="https://colab.research.google.com/github/yeray142/AIML_project/blob/master/notebooks/SVC_without_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML4AI | SVM without "CODE" column 

### Set up for Project

##### Library

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 4.0 MB/s 
     |████████████████████████████████| 81 kB 655 kB/s 
     |████████████████████████████████| 209 kB 59.4 MB/s 
     |████████████████████████████████| 78 kB 7.6 MB/s 
     |████████████████████████████████| 147 kB 55.8 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 112 kB 59.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=6a98dd4e3c7be1480a90e456f819abe71d0f6940135c63a561cb16bf7315b14e
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [3]:
# Modeling
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score, KFold,StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from sklearn.svm import SVC
from sklearn.metrics import log_loss

In [4]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

#### Data Load

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df_train = pd.read_csv("/content/drive/MyDrive/ML4AI/processed_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/ML4AI/processed_test.csv")

In [7]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_test = df_test.drop(['index'],axis =1)

## Data EDA

#### **Basic EDA**

In [8]:
df_train.head()

,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,work_phone,home_phone,...,Age,birth_month,birth_week,ages_employed,employ_month,employ_week,ages_unemployed,unemploy_month,unemploy_week,income_family
0,F,N,N,202500.0,Commercial associate,Higher education,Married,Municipal apartment,0,0,...,38,7.0,1.0,12,0.0,0.0,25,6.0,0.0,101250.0
1,F,N,Y,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,0,1,...,41,10.0,3.0,5,9.0,2.0,35,1.0,0.0,101250.0
2,F,Y,Y,157500.0,State servant,Higher education,Married,House / apartment,0,0,...,41,9.0,0.0,5,10.0,0.0,35,11.0,3.0,78750.0
3,F,N,Y,270000.0,Working,Secondary / secondary special,Married,House / apartment,0,0,...,36,3.0,0.0,13,10.0,1.0,23,4.0,2.0,67500.0
4,F,N,N,315000.0,Working,Secondary / secondary special,Separated,House / apartment,0,0,...,48,9.0,2.0,5,5.0,2.0,42,3.0,3.0,315000.0


In [9]:
df_test.head()

,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,work_phone,home_phone,...,Age,birth_month,birth_week,ages_employed,employ_month,employ_week,ages_unemployed,unemploy_month,unemploy_week,income_family
0,M,Y,N,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,0,1,...,60,1.0,1.0,0,0.0,0.0,60,1.0,1.0,56250.0
1,F,N,Y,135000.0,State servant,Higher education,Married,House / apartment,0,1,...,51,8.0,1.0,23,1.0,2.0,28,7.0,2.0,67500.0
2,F,N,Y,69372.0,Working,Secondary / secondary special,Married,House / apartment,1,1,...,43,1.0,1.0,0,7.0,3.0,42,6.0,2.0,34686.0
3,M,Y,N,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,1,0,...,52,6.0,0.0,6,0.0,1.0,45,5.0,3.0,56250.0
4,F,Y,Y,225000.0,State servant,Higher education,Married,House / apartment,1,0,...,48,6.0,2.0,25,0.0,0.0,23,5.0,1.0,112500.0


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23392 entries, 0 to 23391
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             23392 non-null  object 
 1   car                23392 non-null  object 
 2   reality            23392 non-null  object 
 3   income_total       23392 non-null  float64
 4   income_type        23392 non-null  object 
 5   edu_type           23392 non-null  object 
 6   family_type        23392 non-null  object 
 7   house_type         23392 non-null  object 
 8   work_phone         23392 non-null  int64  
 9   home_phone         23392 non-null  int64  
 10  email              23392 non-null  int64  
 11  occup_type         23392 non-null  object 
 12  family_size        23392 non-null  float64
 13  begin_month        23392 non-null  int64  
 14  credit             23392 non-null  float64
 15  days_unemployed    23392 non-null  int64  
 16  income_unemployed  233

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             10000 non-null  object 
 1   car                10000 non-null  object 
 2   reality            10000 non-null  object 
 3   income_total       10000 non-null  float64
 4   income_type        10000 non-null  object 
 5   edu_type           10000 non-null  object 
 6   family_type        10000 non-null  object 
 7   house_type         10000 non-null  object 
 8   work_phone         10000 non-null  int64  
 9   home_phone         10000 non-null  int64  
 10  email              10000 non-null  int64  
 11  occup_type         10000 non-null  object 
 12  family_size        10000 non-null  float64
 13  begin_month        10000 non-null  int64  
 14  days_unemployed    10000 non-null  int64  
 15  income_unemployed  10000 non-null  float64
 16  Age                1000

In [12]:
df_train.describe()

,income_total,work_phone,home_phone,email,family_size,begin_month,credit,days_unemployed,income_unemployed,Age,birth_month,birth_week,ages_employed,employ_month,employ_week,ages_unemployed,unemploy_month,unemploy_week,income_family
count,2.339200e+04,23392.000000,23392.000000,23392.000000,23392.000000,23392.000000,23392.000000,23392.000000,23392.000000,23392.00000,23392.000000,23392.000000,23392.000000,23392.000000,23392.000000,23392.000000,23392.000000,23392.000000,23392.000000
mean,1.864916e+05,0.222726,0.293519,0.091527,2.201094,26.016288,1.534114,13772.172324,15.465135,43.21875,5.472170,1.490937,5.580711,4.573914,1.257481,37.227257,5.438868,1.513637,100264.108276
std,1.012312e+05,0.416085,0.455384,0.288363,0.918696,16.531449,0.693717,4875.995440,10.263136,11.54970,3.457665,1.116567,6.411067,3.727372,1.170333,13.360431,3.417462,1.125939,72281.859644
min,2.700000e+04,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,6547.000000,1.268406,21.00000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000,0.000000,0.000000,5625.000000
25%,1.215000e+05,0.000000,0.000000,0.000000,2.000000,12.000000,1.000000,9680.000000,8.552947,34.00000,2.000000,0.000000,1.000000,1.000000,0.000000,26.000000,2.000000,0.000000,56250.000000
50%,1.575000e+05,0.000000,0.000000,0.000000,2.000000,24.000000,2.000000,12727.000000,13.307525,42.00000,5.000000,1.000000,4.000000,4.000000,1.000000,34.000000,5.000000,2.000000,78750.000000
75%,2.250000e+05,0.000000,1.000000,0.000000,3.000000,39.000000,2.000000,17032.000000,19.718464,53.00000,9.000000,2.000000,8.000000,8.000000,2.000000,46.000000,8.000000,3.000000,123750.000000
max,1.575000e+06,1.000000,1.000000,1.000000,20.000000,60.000000,2.000000,25152.000000,205.533081,68.00000,11.000000,3.000000,43.000000,11.000000,3.000000,68.000000,11.000000,3.000000,900000.000000


## Data modeling

### Data Preprocessing for construct model

1) Encoding of categorical features

In [13]:
num_col = df_train.dtypes[df_train.dtypes != "object"].index.tolist()
num_col.remove('credit')

cat_col = df_train.dtypes[df_train.dtypes == "object"].index.tolist()

In [14]:
num_col

['income_total',
 'work_phone',
 'home_phone',
 'email',
 'family_size',
 'begin_month',
 'days_unemployed',
 'income_unemployed',
 'Age',
 'birth_month',
 'birth_week',
 'ages_employed',
 'employ_month',
 'employ_week',
 'ages_unemployed',
 'unemploy_month',
 'unemploy_week',
 'income_family']

In [15]:
cat_col

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occup_type']

In [16]:
# StandardScaler
scaler = StandardScaler()
df_train[num_col] = scaler.fit_transform(df_train[num_col])
df_test[num_col] = scaler.fit_transform(df_test[num_col])

In [17]:
# OrdinalEncoding
Encoder = OrdinalEncoder()
df_train[cat_col] = Encoder.fit_transform(df_train[cat_col], df_train['credit'])
df_test[cat_col] = Encoder.fit_transform(df_test[cat_col])


In [18]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23392 entries, 0 to 23391
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             23392 non-null  float64
 1   car                23392 non-null  float64
 2   reality            23392 non-null  float64
 3   income_total       23392 non-null  float64
 4   income_type        23392 non-null  float64
 5   edu_type           23392 non-null  float64
 6   family_type        23392 non-null  float64
 7   house_type         23392 non-null  float64
 8   work_phone         23392 non-null  float64
 9   home_phone         23392 non-null  float64
 10  email              23392 non-null  float64
 11  occup_type         23392 non-null  float64
 12  family_size        23392 non-null  float64
 13  begin_month        23392 non-null  float64
 14  credit             23392 non-null  float64
 15  days_unemployed    23392 non-null  float64
 16  income_unemployed  233

3) Data Split ⇒ We need to split with stratified split

In [19]:
X_train, X_val, y_train, y_val = train_test_split(df_train.loc[:,df_train.columns != 'credit'], df_train['credit'], test_size = 0.2)

In [20]:
X_train.head()

,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,work_phone,home_phone,...,Age,birth_month,birth_week,ages_employed,employ_month,employ_week,ages_unemployed,unemploy_month,unemploy_week,income_family
9869,0.0,1.0,0.0,-0.064128,0.0,4.0,1.0,1.0,-0.535301,-0.644567,...,-1.057951,0.441877,0.455928,-0.402549,0.650898,1.488940,-0.765505,1.627304,1.320138,-0.142004
1946,0.0,0.0,1.0,-0.286397,2.0,4.0,1.0,1.0,-0.535301,-0.644567,...,-0.884782,-1.582654,-1.335315,-0.090581,0.919189,-1.074487,-0.765505,-0.713664,-1.344362,-0.842402
21513,0.0,0.0,1.0,-0.286397,4.0,4.0,1.0,1.0,-0.535301,1.551429,...,-0.451861,-0.714998,1.351549,-0.870500,0.382606,0.634464,0.057839,0.749441,-1.344362,-0.660818
17835,0.0,0.0,0.0,-0.508665,4.0,1.0,1.0,1.0,1.868108,-0.644567,...,-0.625030,0.731096,0.455928,0.221386,-0.153976,0.634464,-0.690655,-0.713664,-1.344362,-0.453292
16859,1.0,1.0,1.0,-1.086563,4.0,4.0,1.0,1.0,1.868108,1.551429,...,0.500565,-0.136560,-1.335315,0.065402,-0.422267,1.488940,0.432087,-1.006285,-0.456196,-0.857967


In [21]:
y_train

9869     1.0
1946     2.0
21513    0.0
17835    2.0
16859    0.0
        ... 
7939     1.0
4256     2.0
17560    1.0
18366    1.0
15188    2.0
Name: credit, Length: 18713, dtype: float64

### Modeling

- Optuna

In [22]:
def objective(trial: Trial) -> float:
    params_svc = {
        "random_state": 42,
        'kernel': 'rbf',
        'C' : trial.suggest_loguniform('C', 1e-5,1e+5),
        'gamma' : trial.suggest_loguniform('gamma', 1e-5,1e+5),
        'probability' : True,
    }
    model = sklearn.svm.SVC(**params_svc)
    model.fit(
        X_train,
        y_train
    )

    svc_pred = model.predict_proba(X_val)
    log_score = log_loss(y_val, svc_pred)
    
    return log_score

In [23]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="svc_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2022-12-11 04:54:32,466] A new study created in memory with name: svc_parameter_opt
[I 2022-12-11 05:04:39,296] Trial 0 finished with value: 0.8892866254390058 and parameters: {'C': 0.05564180225431373, 'gamma': 32147.193482816965}. Best is trial 0 with value: 0.8892866254390058.
[I 2022-12-11 05:20:26,467] Trial 1 finished with value: 0.8598380892521893 and parameters: {'C': 208.90047049266641, 'gamma': 9.695826644515218}. Best is trial 1 with value: 0.8598380892521893.
[I 2022-12-11 05:22:30,405] Trial 2 finished with value: 0.920378619681429 and parameters: {'C': 0.0003632339256943143, 'gamma': 0.0003630322466779861}. Best is trial 1 with value: 0.8598380892521893.
[I 2022-12-11 05:24:17,317] Trial 3 finished with value: 0.8892735127393657 and parameters: {'C': 3.809220577048033e-05, 'gamma': 4589.458612326462}. Best is trial 1 with value: 0.8598380892521893.
[I 2022-12-11 05:38:27,987] Trial 4 finished with value: 0.8720559267177778 and parameters: {'C': 10.260065124896771, 'gam

Best Score: 0.8167809437549748
Best trial: {'C': 0.011026112761510002, 'gamma': 1.7683340776662542}


In [24]:
study.best_params['C']

0.011026112761510002

In [25]:
study.best_params['gamma']

1.7683340776662542

--- 
K Fold

In [26]:
p_optuna = {
    'kernel' : 'rbf',
    "random_state": 42,
    'C' : study.best_params['C'],
    'gamma' : study.best_params['gamma'],
    'probability' : True,
}

In [27]:
def stratified_kfold_svc(p, n_fold, X, y, X_test):
    # Declaring Stratified K-Fold:
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    
    # Initializing arrays:
    svc_oof = np.zeros((X.shape[0], 3))
    svc_preds = np.zeros((X_test.shape[0], 3))

    # Main loop:
    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============")
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        # Model fit:
        model = SVC(**p)
        model.fit(X_train, y_train)

        svc_oof[valid_idx] = model.predict_proba(X_valid) # Infers the class probability of train dataset
        svc_preds += model.predict_proba(X_test) / n_fold # Infers the class probability of test dataset
        print(f"Log Loss Score: {log_loss(y_valid, svc_oof[valid_idx]):.5f}")

    log_score = log_loss(y, svc_oof)
    print(f"\nLog Loss Score: {log_score:.5f}")

    return svc_oof, svc_preds

In [28]:
svc_oof_optuna, svc_preds_optuna = stratified_kfold_svc(p_optuna, 10, df_train.drop("credit", axis=1),df_train['credit'] , df_test)

============ Fold 0 ============
Log Loss Score: 0.79431
============ Fold 1 ============
Log Loss Score: 0.80738
============ Fold 2 ============
Log Loss Score: 0.78580
============ Fold 3 ============
Log Loss Score: 0.78709
============ Fold 4 ============
Log Loss Score: 0.79080
============ Fold 5 ============
Log Loss Score: 0.78142
============ Fold 6 ============
Log Loss Score: 0.80974
============ Fold 7 ============
Log Loss Score: 0.79085
============ Fold 8 ============
Log Loss Score: 0.79711
============ Fold 9 ============
Log Loss Score: 0.79304

Log Loss Score: 0.79375


In [29]:
p_grid = {
    'kernel' : 'rbf',
    "random_state": 42,
    'C' : 10.0,
    'gamma' : 0.1,
    'probability' : True,
}

In [30]:
svc_oof_grid, svc_preds_grid = stratified_kfold_svc(p_grid, 10, df_train.drop("credit", axis=1),df_train['credit'] , df_test)

============ Fold 0 ============
Log Loss Score: 0.77124
============ Fold 1 ============
Log Loss Score: 0.77585
============ Fold 2 ============
Log Loss Score: 0.74916
============ Fold 3 ============
Log Loss Score: 0.76008
============ Fold 4 ============
Log Loss Score: 0.76361
============ Fold 5 ============
Log Loss Score: 0.75412
============ Fold 6 ============
Log Loss Score: 0.77112
============ Fold 7 ============
Log Loss Score: 0.75922
============ Fold 8 ============
Log Loss Score: 0.77216
============ Fold 9 ============
Log Loss Score: 0.77094

Log Loss Score: 0.76475


In [31]:
df_submission = pd.read_csv('/content/drive/MyDrive/ML4AI/sample_submission.csv')

In [32]:
df_submission.set_index("index", inplace=True)
df_submission[:] = svc_preds_grid
df_submission.head(20)

,0,1,2
index,,,
26457,0.051733,0.115838,0.832429
26458,0.452527,0.224731,0.322743
26459,0.103964,0.143715,0.752321
26460,0.101596,0.132846,0.765558
26461,0.092268,0.445091,0.462642
26462,0.055290,0.185627,0.759083
26463,0.260835,0.154900,0.584265
26464,0.085856,0.191769,0.722375
26465,0.089147,0.115241,0.795612


In [33]:
df_submission.to_csv('/content/drive/MyDrive/ML4AI/SVC without code.csv',index=False)